In [1]:
import os
from pathlib import Path
import pandas as pd
import requests
from bs4 import BeautifulSoup

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

import time

In [2]:
DATA_PATH = Path.cwd().parent / "runtime" / "data" / "train_features" / "train_features"
url = "https://www.veganrecipeclub.org.uk/recipes/"
SCROLL_PAUSE_TIME = 3

In [3]:
def parent(path_):
    return os.path.abspath(os.path.join(path_, os.pardir))

In [4]:
def get_html(url):  
    #get chrome driver path
    driver_path = os.path.join(parent(os.getcwd()), 'chromedriver', 'chromedriver.exe.')
    #create instance of Chrome driver
    driver = Chrome(executable_path=driver_path)
    
    driver.get(url)

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    #load page source
    html = driver.page_source
    #close driver
    driver.close()
    return html

def parse_url(html):
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('div', {'class': 'category-products'})
    rows = table.find_all('div', {'class': 'product-info'})

    projects = []
    for row in rows:
        cols = row.find_all('h2')
        projects.append({
            'title': cols[0].text,
            'url': cols[0].a['href']
            })
    return projects

In [5]:
html = get_html(url)
html

'<html class="no-js" lang="en-GB"><head>\n\t\t<meta charset="UTF-8">\n\t\t<title> Advance recipe search - Vegan Recipe Club</title>\n\t\t<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n\t\t<meta name="apple-mobile-web-app-capable" content="yes">\n\t\t<meta name="apple-mobile-web-app-status-bar-style" content="black">\n\t\t<meta name="viewport" content="width=device-width, initial-scale=1.0">\n\t\t<meta name="description" content="Hundreds of tried and tested vegan recipes, with product tips and news! We’ve got convenience-style supermarket meals and beginner recipes, right up to gourmet. Search for recipes by courses, seasons, celebrations, dietary requirements, international cuisine, guest chefs and much more.">\n\n\t\t\t<script async="" src="https://connect.facebook.net/en_US/fbevents.js"></script><script async="" src="https://www.google-analytics.com/analytics.js"></script><script async="" src="//www.googletagmanager.com/gtm.js?id=GTM-KFW23HK"></script><script type="